In [2]:
import carla
import time
import sys
sys.path.append('/home/kaustubh/CARLA_15/PythonAPI/carla/')

client = carla.Client("localhost", 2000)
world = client.get_world()

In [3]:
vehicle = world.get_actors().filter("*vehicle*")[0]
gnss = world.get_actors().filter("*gnss*")[0]
print(vehicle, gnss)

Actor(id=48, type=vehicle.volkswagen.t2_2021) Actor(id=50, type=sensor.other.gnss)


In [ ]:
gnss_data = None

def gnss_callback(data):
    global gnss_data
    gnss_data = data

gnss.listen(gnss_callback)

# Wait until data comes in
while gnss_data is None:
    time.sleep(0.1)

lat = gnss_data.latitude
lon = gnss_data.longitude
alt = gnss_data.altitude

carla_loc = vehicle.get_transform().location
print(f"GNSS = ({lat}, {lon}, {alt})")
print(f"Location = ({carla_loc.x}, {carla_loc.y}, {carla_loc.z})")


GNSS = (-0.00021983009746406879, -0.0005807149257589768, 1.996480107307434)
Location = (-64.6448745727539, 24.47144889831543, -0.003519897349178791)


In [13]:
# Wait until data comes in
while gnss_data is None:
    time.sleep(0.1)

lat = gnss_data.latitude
lon = gnss_data.longitude
alt = gnss_data.altitude

carla_loc = vehicle.get_transform().location
print(f"GNSS = ({lat}, {lon}, {alt})")
print(f"Location = ({carla_loc.x}, {carla_loc.y}, {carla_loc.z})")

GNSS = (-0.00022121913406181193, -6.453951494285637e-05, 1.9992948770523071)
Location = (-7.1847381591796875, 24.626001358032227, -0.0007050705025903881)


In [ ]:
GNSS = (-0.00021983009746406879, -0.0005807149257589768, 1.996480107307434)
Location = (-64.6448745727539, 24.47144889831543, -0.003519897349178791)

GNSS = (-0.00022035828743582897, -0.0003889849722697718, 1.9989652633666992)
Location = (-43.30065155029297, 24.530176162719727, -0.001034431392326951)

GNSS = (-0.00022079931669338748, -0.00022790756398132856, 1.9992910623550415)
Location = (-25.37077522277832, 24.579267501831055, -0.0007089614518918097)

GNSS = (-0.00022121913406181193, -6.453951494285637e-05, 1.9992948770523071)
Location = (-7.1847381591796875, 24.626001358032227, -0.0007050705025903881)

In [8]:
map = world.get_map()

In [11]:
print(map.get_spawn_points()[0])

Transform(Location(x=-64.644844, y=24.471010, z=0.600000), Rotation(pitch=0.000000, yaw=0.159198, roll=0.000000))


In [ ]:
start_location = world.get_map().get_spawn_points()[0].location
geo = world.get_map().transform_to_geolocation(start_location)

print(start_location)
print(geo)

Location(x=-64.644844, y=24.471010, z=0.600000)
GeoLocation(latitude=-0.000220, longitude=-0.000581, altitude=0.600000)


In [14]:
def gnss_to_location(lat, lon):
    x = -14418.6285 * lat + 111279.5690 * lon - 3.19252014
    y = -109660.6210 * lat +    4.33686914 * lon + 0.367254638
    return carla.Location(x=x, y=y, z=0.0)


In [22]:
# 1. Convert start and goal GNSS to CARLA locations
start_lat = -0.000220
start_lon = -0.000581
goal_lat = -0.00022121913406181193
goal_lon =   -6.453951494285637
start_loc = gnss_to_location(start_lat, start_lon)
goal_loc  = gnss_to_location(goal_lat,  goal_lon)

# 2. Plan the route
from agents.navigation.global_route_planner import GlobalRoutePlanner
grp = GlobalRoutePlanner(world.get_map(), sampling_resolution=2.0)
route = grp.trace_route(start_loc, goal_loc)

# 3. Visualize
for wp, _ in route:
    world.debug.draw_point(wp.transform.location + carla.Location(z=0.5),
                           size=0.2,
                           color=carla.Color(0,255,0),
                           life_time=30.0)


In [8]:
for actor in world.get_actors().filter("*imu*"):
    x = actor
    print(actor)

Actor(id=31, type=sensor.other.imu)


In [9]:
x.listen()

ArgumentError: Python argument types in
    Sensor.listen(ServerSideSensor)
did not match C++ signature:
    listen(carla::client::Sensor {lvalue}, boost::python::api::object callback)

In [46]:
x = vehicle.get_transform().rotation
print(x)

Rotation(pitch=0.000068, yaw=98.611740, roll=0.000000)


In [28]:
print(type(x.rotation.yaw))

<class 'float'>


In [30]:
import math
def euler_to_quaternion():
    yaw, roll = 0, 0
    pitch= vehicle.get_transform().rotation.yaw
    cy = math.cos(yaw * 0.5)
    sy = math.sin(yaw * 0.5)
    cp = math.cos(pitch * 0.5)
    sp = math.sin(pitch * 0.5)
    cr = math.cos(roll * 0.5)
    sr = math.sin(roll * 0.5)

    return {
        "w": cr * cp * cy + sr * sp * sy,
        "x": sr * cp * cy - cr * sp * sy,
        "y": cr * sp * cy + sr * cp * sy,
        "z": cr * cp * sy - sr * sp * cy
    }

In [40]:
x = euler_to_quaternion()

In [38]:
print(x, y, z, w)

w x y z


In [41]:
print(x)

{'w': 0.5738374605835068, 'x': 0.0, 'y': -0.8189692111618557, 'z': 0.0}


In [43]:
print(x['w'])

0.5738374605835068


In [6]:
for ac in world.get_blueprint_library().filter("*lane*"):
    print(ac)

ActorBlueprint(id=sensor.other.lane_invasion,tags=[lane_invasion, other, sensor])
